In [ ]:
# Text text processing library
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import itertools as it
from models import *
from helpers import *
# import main
import matplotlib.pyplot as plt
import spacy
import time
import itertools as it
MAX_LEN = 20
MIN_FREQ = 5

In [ ]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de)

# only target needs BOS/EOS:
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) 

train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)

In [ ]:
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)

pred_set = []
for i, line in enumerate(open("source_test.txt"), 1):
    words = line.split()# [:-1]
    pred_set.append([DE.vocab.stoi[s] for s in words])

train_iter, val_iter, test_iter = data.BucketIterator.splits((train, val, test), batch_size=32, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))

In [ ]:
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=32, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))
bs_encoder = BaseEncoder(DE, hidden_size=500, num_layers=4, word_features=500)
bs_decoder = BaseDecoder(EN, hidden_size=500, num_layers=4, word_features=500)
trainer = NMTTrainer([bs_encoder, bs_decoder], DE, EN, lrn_rate=0.7, 
                     lrn_decay='adaptive', reverse_enc_input=False)
evaluator = NMTEvaluator([bs_encoder, bs_decoder], DE, EN, reverse_enc_input=False)
trainer.train(train_iter, verbose=True, le=evaluator, val_iter=val_iter)

In [ ]:
bs_encoder = BaseEncoder(DE, hidden_size=650, num_layers=4, word_features=650, 
                         bidirectional=True, dropout=0.35)
# TODO: decide whether to add dropout to output states of encoder!
at_decoder = AttnDecoder(EN, hidden_size=650, num_layers=4, word_features=650, dropout=0.35,
                         tie_weights=True, enc_linear=650,
                         enc_bidirectional=True)
evaluator = NMTEvaluator([bs_encoder, at_decoder], DE, EN, attention=True,
                        record_attention=False, cuda=True)
evaluator.evaluate(test_iter)

In [ ]:
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=32, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))
bs_encoder = BaseEncoder(DE, hidden_size=650, num_layers=4, word_features=650, 
                         bidirectional=True, dropout=0.35)
# TODO: decide whether to add dropout to output states of encoder!
at_decoder = AttnDecoder(EN, hidden_size=650, num_layers=4, word_features=650, dropout=0.35,
                         tie_weights=True, enc_linear=650,
                         enc_bidirectional=True)
trainer = NMTTrainer([bs_encoder, at_decoder], DE, EN, lrn_rate=1.2, 
                     lrn_decay='adaptive', attention=True,
                     clip_norm=5, lrn_decay_force=8, cuda=True)
evaluator = NMTEvaluator([bs_encoder, at_decoder], DE, EN, attention=True,
                        record_attention=False, cuda=True)
trainer.train(train_iter, verbose=True, le=evaluator, val_iter=val_iter,
              save_model_fn='attn_med_5', num_iter=20)
# KEYXX

In [ ]:
# Grid search example
cnt = 0
for hs, tw, force, drop in it.product([200,400,600],[False, True],
                                      [4,8],[0.2, 0.4]):
    print(hs, tw, force, drop)
    train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=32, device=-1,
                                                      repeat=False, sort_key=lambda x: len(x.src))
    bs_encoder = BaseEncoder(DE, hidden_size=hs, num_layers=4, word_features=hs, 
                             dropout=drop)
    # TODO: decide whether to add dropout to output states of encoder!
    at_decoder = AttnDecoder(EN, hidden_size=hs, num_layers=4, word_features=hs, dropout=drop,
                             tie_weights=tw)
    trainer = NMTTrainer([bs_encoder, at_decoder], DE, EN, lrn_rate=1.0, 
                         lrn_decay='adaptive', attention=True,
                         clip_norm=5, lrn_decay_force=force)
    evaluator = NMTEvaluator([bs_encoder, at_decoder], DE, EN, attention=True,
                            record_attention=False)
    trainer.train(train_iter, verbose=True, le=evaluator, val_iter=val_iter,
                  save_model_fn='attn_grid_%d' % cnt, num_iter=10)
    cnt += 1

In [ ]:
# Load a model and continue training:
ld_enc, ld_dec = load_checkpoint('saved_models/attn_med_1.epoch_8.ckpt.tar')
ld_encoder = BaseEncoder(DE, hidden_size=400, num_layers=4, word_features=400, 
                         dropout=0.2)
ld_decoder = AttnDecoder(EN, hidden_size=400, num_layers=4, word_features=400, 
                         dropout=0.2)
set_parameters(ld_encoder, ld_enc)
set_parameters(ld_decoder, ld_dec)
evaluator = NMTEvaluator([ld_encoder, ld_decoder], DE, EN, attention=True,
                        record_attention=False)
print(evaluator.evaluate(val_iter))
trainer = NMTTrainer([ld_encoder, ld_decoder], DE, EN, lrn_rate=0.5, 
                     lrn_decay='adaptive', attention=True,
                     clip_norm=5, lrn_decay_force=2)
trainer.train(train_iter, verbose=True, le=evaluator, val_iter=val_iter,
              save_model_fn='attn_med_1b', init_parameters=False)

In [ ]:
print(evaluator.evaluate(val_iter))
print(evaluator.evaluate(test_iter))

In [ ]:
# Load a saved model and produce predictions
# save_checkpoint(bs_encoder, at_decoder, filename='saved_models/attn_big_0.epoch_10.ckpt.tar')
ld_enc, ld_dec = load_checkpoint('saved_models/attn_med_5.epoch_10.ckpt.tar')
ld_encoder = BaseEncoder(DE, hidden_size=650, num_layers=4, word_features=650, 
                         bidirectional=True, dropout=0.35)
ld_decoder = AttnDecoder(EN, hidden_size=650, num_layers=4, word_features=650, dropout=0.35,
                         tie_weights=False, enc_linear=650,
                         enc_bidirectional=True)
set_parameters(ld_encoder, ld_enc)
set_parameters(ld_decoder, ld_dec)
evaluator = NMTEvaluator([ld_encoder, ld_decoder], DE, EN, attention=True,
                        record_attention=False)
print(evaluator.evaluate(val_iter))
print(evaluator.evaluate(test_iter)) # KEYYY
evaluator.predict(pred_set, fn='predictions_big_2_100.txt',beam_size=100, ignore_eos=True)

In [ ]:
# Visualize attn distribution
attn_list = list(evaluator.attns_log[17][i][1] for i in range(4))
evaluator.visualize_attn(*attn_list)